<a href="https://colab.research.google.com/github/Anitagg1/Phyton-Workshop/blob/main/scraping_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

## Extract ASIN from the url of item products

In [ ]:
# function to extract ASIn from URL


def get_asin(my_df):
    temp_asin = my_df
    temp_url = temp_asin.loc[:,'url']
    temp_url_df = pd.DataFrame(temp_url)

    for row in range(len(temp_url_df)):
        # divide the full url into three parts using the function partition()
        my_text = temp_url_df.loc[row, 'url'] 
        division_tag1 = str(r"%2Fdp%2F")
        division_tag2 = str('/dp/')
        if division_tag1 in my_text:
            right_text = my_text.partition(division_tag1)[2] 
        else:
            right_text = my_text.partition(division_tag2)[2] 
        temp_url_df.loc[row, 'right_text'] = right_text

        # divide the target part ('right_text') into three parts using the function partition()
        my_text2 = temp_url_df.loc[row, 'right_text'] 
        division_tag1 = str(r"%2F")
        division_tag2 = str('/')
        if division_tag1 in my_text2:
            target_text = my_text2.partition(division_tag1)[0] 
        else:
            target_text = my_text2.partition(division_tag2)[0] 

        # add the final target ('asin') in the new column 'ASIN'
        temp_url_df.loc[row, 'ASIN'] = target_text
        temp_url_df = temp_url_df[['url','ASIN']]

    # check if ASIN is correctly added to the dataframe
    print(temp_url_df.head())
    out_df = my_df.merge(temp_url_df, on='url')

    return out_df 


# source reference: https://stackoverflow.com/questions/8162021/analyzing-string-input-until-it-reaches-a-certain-letter-on-python
# relevant functions: split(), partition()

In [ ]:
# run the function 'get_asin' to extract asin from the column 'url' in the summary dataframe 'df_cleaned'
df = pd.read_json('./output/search_output/Mouse_summary.jsonl')

df_asin = get_asin(df)

# check the updated df with the column 'ASIN'
df_asin.head()

# Import document with ASIN
Its the same document that the search output but I created a new column on excel and extracted the asin at the end. 
When I import this csv document on python, its somehow divided by semicolon.
IF you already manage to scrape the asin, then just import the search output json and follow almost the same steps to clean it.

In [ ]:
# Import document with ASIN
df = pd.read_csv('/content/asin.csv', sep=';')

In [ ]:
df

,title,url,rating,review_count,price,ASIN
0,Mouse Pad - Oriental Carpet Woven Fabric Mouse...,https://www.amazon.com/Mouse-Pad-Oriental-Mous...,4.4 out of 5 stars,2,$18.95,1578507596
1,Mouse Pad - Oriental Carpet Woven Fabric Mouse...,https://www.amazon.com/Mouse-Pad-Oriental-Mous...,4.4 out of 5 stars,2,$18.95,1578507596
2,Oriental Carpet Mouse Pad - Authentic Woven Ca...,https://www.amazon.com/Oriental-Carpet-Mousepa...,4.9 out of 5 stars,25,$19.95,157850550X
3,"Belkin WaveRest Gel Mouse Pad, Black (F8E262-BLK)",https://www.amazon.com/Belkin-WaveRest-Mouse-B...,4.6 out of 5 stars,"18,052",$8.00,B00000JRRD
4,Fellowes 58024 Medium Mouse Pad (Black),https://www.amazon.com/Fellowes-58024-Medium-M...,4.7 out of 5 stars,"7,848",NaN,B0000365QL
...,...,...,...,...,...,...
7475,RAPOO N100 Wired Mouse – Corded USB Mouse for ...,https://www.amazon.com/gp/slredirect/picassoRe...,4.2 out of 5 stars,26,$6.99,B07WJYSHFF
7476,RAPOO N100 Wired Mouse – Corded USB Mouse for ...,https://www.amazon.com/gp/slredirect/picassoRe...,4.2 out of 5 stars,26,$6.99,B07WJYSHFF
7477,RAPOO N100 Wired Mouse – Corded USB Mouse for ...,https://www.amazon.com/gp/slredirect/picassoRe...,4.2 out of 5 stars,26,$6.99,B07WJYSHFF
7478,RAPOO N100 Wired Mouse – Corded USB Mouse for ...,https://www.amazon.com/gp/slredirect/picassoRe...,4.2 out of 5 stars,26,$6.99,B07WJYSHFF


## Create the prod_type column

In [ ]:
# Create the prod_type column
df.loc[:, 'product_type'] = 'Mouse'

In [ ]:
# Check datatypes
df.dtypes

title           object
url             object
rating          object
review_count    object
price           object
ASIN            object
product_type    object
dtype: object

In [ ]:
df.shape

(7480, 7)

## Missing values

I haven't done anything with the NaN values on this code. 

In [ ]:
# Check missing values

df.isnull().sum()

title             0
url               0
rating          510
review_count    510
price           500
ASIN              0
product_type      0
dtype: int64

## Rating

In [ ]:
# Split the string into 2 parts
df.rating.str.split(' ', n = 1)[0]

['4.4', 'out of 5 stars']

In [ ]:
# Create a loop that checks the the "rating" column and takes the first split
for i in range(len(df.rating)):
    try:
        df.loc[i,'rating'] = df.loc[i,'rating'].split(' ')[0]
    except:
        continue
        #print('Sorry lets move on')

In [ ]:
# Convert string values into float
df.rating = df.rating.astype('float')

## Price

In [ ]:
# Prepare the Data on ["price"] for the analysis

# 1 Replace non compatible characters
price_string = df["price"].str.replace("$", "")

# 2 Replace the "," and convert the string into float
price_num = price_string.str.replace(',', '').astype(float)

# 3 Substitute columns or create a new one
df["price"] = price_num

In [ ]:
type(df["review_count"][0])

str

In [ ]:
# Prepare the Data on ["review_count"] for the analysis

# 1 Replace the "," and convert the string into float
review_count_num = df.review_count.str.replace(',', '').astype(float)

# 2 Substitute columns or create a new one
df["review_count"] = review_count_num

# Sponsored products
We consider that all the products that are duplicated are sponsored products

0 = not sponsored
1 = sponsored

In [ ]:
# Create a new column "sponsored" with the duplicate products
df['sponsored'] = df['title'].duplicated(keep=False).astype(int).astype(str)


In [ ]:
df

,title,url,rating,review_count,price,ASIN,product_type,sponsored
0,Mouse Pad - Oriental Carpet Woven Fabric Mouse...,https://www.amazon.com/Mouse-Pad-Oriental-Mous...,4.4,2.0,18.95,1578507596,Mouse,1
1,Mouse Pad - Oriental Carpet Woven Fabric Mouse...,https://www.amazon.com/Mouse-Pad-Oriental-Mous...,4.4,2.0,18.95,1578507596,Mouse,1
2,Oriental Carpet Mouse Pad - Authentic Woven Ca...,https://www.amazon.com/Oriental-Carpet-Mousepa...,4.9,25.0,19.95,157850550X,Mouse,0
3,"Belkin WaveRest Gel Mouse Pad, Black (F8E262-BLK)",https://www.amazon.com/Belkin-WaveRest-Mouse-B...,4.6,18052.0,8.00,B00000JRRD,Mouse,0
4,Fellowes 58024 Medium Mouse Pad (Black),https://www.amazon.com/Fellowes-58024-Medium-M...,4.7,7848.0,NaN,B0000365QL,Mouse,0
...,...,...,...,...,...,...,...,...
7475,RAPOO N100 Wired Mouse – Corded USB Mouse for ...,https://www.amazon.com/gp/slredirect/picassoRe...,4.2,26.0,6.99,B07WJYSHFF,Mouse,1
7476,RAPOO N100 Wired Mouse – Corded USB Mouse for ...,https://www.amazon.com/gp/slredirect/picassoRe...,4.2,26.0,6.99,B07WJYSHFF,Mouse,1
7477,RAPOO N100 Wired Mouse – Corded USB Mouse for ...,https://www.amazon.com/gp/slredirect/picassoRe...,4.2,26.0,6.99,B07WJYSHFF,Mouse,1
7478,RAPOO N100 Wired Mouse – Corded USB Mouse for ...,https://www.amazon.com/gp/slredirect/picassoRe...,4.2,26.0,6.99,B07WJYSHFF,Mouse,1


## Change order on the columns

In [ ]:
# print the columns as a list 
cols = list(df.columns.values)
cols

['title',
 'url',
 'rating',
 'review_count',
 'price',
 'ASIN',
 'product_type',
 'sponsored']

In [ ]:
# Drop unnecesary colummns - url
df = df.drop(columns=['url'])

In [ ]:
# Rearrange colummns order
df = df[['ASIN', 'title', 'price', 'rating', 'review_count','product_type', 'sponsored' ]]


In [ ]:
df

,ASIN,title,price,rating,review_count,product_type,sponsored
0,1578507596,Mouse Pad - Oriental Carpet Woven Fabric Mouse...,18.95,4.4,2.0,Mouse,1
1,1578507596,Mouse Pad - Oriental Carpet Woven Fabric Mouse...,18.95,4.4,2.0,Mouse,1
2,157850550X,Oriental Carpet Mouse Pad - Authentic Woven Ca...,19.95,4.9,25.0,Mouse,0
3,B00000JRRD,"Belkin WaveRest Gel Mouse Pad, Black (F8E262-BLK)",8.00,4.6,18052.0,Mouse,0
4,B0000365QL,Fellowes 58024 Medium Mouse Pad (Black),NaN,4.7,7848.0,Mouse,0
...,...,...,...,...,...,...,...
7475,B07WJYSHFF,RAPOO N100 Wired Mouse – Corded USB Mouse for ...,6.99,4.2,26.0,Mouse,1
7476,B07WJYSHFF,RAPOO N100 Wired Mouse – Corded USB Mouse for ...,6.99,4.2,26.0,Mouse,1
7477,B07WJYSHFF,RAPOO N100 Wired Mouse – Corded USB Mouse for ...,6.99,4.2,26.0,Mouse,1
7478,B07WJYSHFF,RAPOO N100 Wired Mouse – Corded USB Mouse for ...,6.99,4.2,26.0,Mouse,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7480 entries, 0 to 7479
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ASIN          7480 non-null   object 
 1   title         7480 non-null   object 
 2   price         6980 non-null   float64
 3   rating        6970 non-null   float64
 4   review_count  6970 non-null   float64
 5   product_type  7480 non-null   object 
 6   sponsored     7480 non-null   object 
dtypes: float64(3), object(4)
memory usage: 409.2+ KB


## Export as csv

In [ ]:
# Export as an csv
df.to_csv('mouse_summary.csv', index=False)